In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from shapely.geometry import Point
from tqdm.auto import tqdm
tqdm.pandas()
import os

In [ ]:
# --- Configuración de Rutas y Nombres de Archivo (Globales para este Notebook) ---
# Directorios
RUTA_DATOS_INPUT_ORIGINALES = './datos/'
RUTA_DATOS_CENSO_CSV_DIR = './datos/censo/'
RUTA_MGN_SHAPEFILES_DIR = './datos/mgn/conjunto_de_datos/'
RUTA_DENUE_RAW_DIR = './datos/denue_raw/'
RUTA_OSM_DATA_DIR = './datos/osm_data/'
RUTA_IMU_CSV_DIR = './datos/imu_data/'
RUTA_DATOS_OUTPUT_GENERAL = './datos/output/' # Para el JSON de SCIAN y el resultado final

# Archivo de entrada principal para este notebook
ARCHIVO_DIM_TIENDA_TEST = RUTA_DATOS_INPUT_ORIGINALES + 'DIM_TIENDA_TEST.csv'

# Archivos de datos externos (asumiendo nombres y ubicaciones de scripts anteriores)
# Censo
ARCHIVO_CENSO_NL = RUTA_DATOS_CENSO_CSV_DIR + 'nuevo_leon_censo_ageb.csv'
ARCHIVO_CENSO_TAM = RUTA_DATOS_CENSO_CSV_DIR + 'tamaulipas_censo_ageb.csv'
COL_CENSO_SCITEL_ENTIDAD = 'ENTIDAD' # Ajustar si es diferente en tus CSV de SCITEL
COL_CENSO_SCITEL_MUNICIPIO = 'MUN'
COL_CENSO_SCITEL_LOCALIDAD = 'LOC'
COL_CENSO_SCITEL_AGEB = 'AGEB'
COL_CENSO_CVEGEO_CONSTRUIDA = 'CVEGEO_CONSTRUIDO'

# MGN (AGEBs)
SHAPEFILE_AGEB_NACIONAL = RUTA_MGN_SHAPEFILES_DIR + '00a.shp' 
COL_SHP_CVEGEO = 'CVEGEO' 
CRS_SHAPEFILE_AGEB = 'EPSG:6372'

# DENUE
ARCHIVOS_DENUE_CSV_LISTA = [
    RUTA_DENUE_RAW_DIR + 'denue_nuevo_leon.csv', 
    RUTA_DENUE_RAW_DIR + 'denue_tamaulipas.csv'
]
COL_DENUE_LAT = 'latitud'
COL_DENUE_LON = 'longitud'
COL_DENUE_SCIAN_CODE = 'codigo_act'
JSON_CATEGORIAS_SCIAN = RUTA_DATOS_OUTPUT_GENERAL + 'categorias_pdi_scian_final.json'

# OSM
SHP_OSM_POIS_FILE = RUTA_OSM_DATA_DIR + 'gis_osm_pois_a_free_1.shp' 
SHP_OSM_ROADS_FILE = RUTA_OSM_DATA_DIR + 'gis_osm_roads_free_1.shp'
COL_OSM_POI_FCLASS = 'fclass'
COL_OSM_ROAD_FCLASS = 'fclass'

# IMU
ARCHIVO_IMU_CSV = RUTA_IMU_CSV_DIR + 'indice_marginacion_urbana_ageb_2020.csv'
COL_IMU_CVEGEO_KEY = 'CVE_AGEB' # Clave en el CSV del IMU
COL_IMU_INDEX_VALUE = 'IM_2020'
COL_IMU_GRADE_VALUE = 'GM_2020'
COL_IMU_NAT_INDEX_VALUE = 'IMN_2020'

# Parámetros de Análisis
RADIOS_M = [200, 500, 1000] # Radios para conteos de PDIs

# Archivo de Salida para este notebook
ARCHIVO_SALIDA_TEST_ENRIQUECIDO_GPKG = RUTA_DATOS_OUTPUT_GENERAL + 'tiendas_TEST_OFICIAL_enriquecido.gpkg'
LAYER_SALIDA_TEST_ENRIQUECIDO = 'tiendas_test_oficial_enriquecido'

# CRS Globales
CRS_PROYECTADO_OBJETIVO = CRS_SHAPEFILE_AGEB # EPSG:6372
CRS_GEOGRAFICO_WGS84 = 'EPSG:4326'

# Crear directorios de output si no existen
if not os.path.exists(RUTA_DATOS_OUTPUT_GENERAL): os.makedirs(RUTA_DATOS_OUTPUT_GENERAL)
# --- Fin de Configuración ---

print("--- Iniciando Enriquecimiento de DIM_TIENDA_TEST.csv ---")


In [ ]:
# --- PASO 0: Cargar DIM_TIENDA_TEST.csv y convertir a GeoDataFrame ---
try:
    df_test_original = pd.read_csv(ARCHIVO_DIM_TIENDA_TEST)
    print(f"Cargado '{ARCHIVO_DIM_TIENDA_TEST}': {len(df_test_original)} tiendas de test.")
    
    # Verificar columnas de coordenadas
    if not {'LATITUD_NUM', 'LONGITUD_NUM'}.issubset(df_test_original.columns):
        raise ValueError("Columnas 'LATITUD_NUM' y 'LONGITUD_NUM' no encontradas en DIM_TIENDA_TEST.csv")

    tiendas_test_gdf = gpd.GeoDataFrame(
        df_test_original,
        geometry=gpd.points_from_xy(df_test_original.LONGITUD_NUM, df_test_original.LATITUD_NUM),
        crs=CRS_GEOGRAFICO_WGS84 
    )
    # Reproyectar al CRS objetivo
    tiendas_test_gdf = tiendas_test_gdf.to_crs(CRS_PROYECTADO_OBJETIVO)
    print(f"GeoDataFrame de tiendas TEST creado y reproyectado a {tiendas_test_gdf.crs}. Shape: {tiendas_test_gdf.shape}")

except FileNotFoundError:
    print(f"Error: No se encontró '{ARCHIVO_DIM_TIENDA_TEST}'.")
    raise
except Exception as e:
    print(f"Error al cargar o convertir DIM_TIENDA_TEST.csv: {e}")
    raise

In [ ]:
# === PASO 1: Integración de Datos del Censo (similar a 04_...) ===
print("\n--- Iniciando Integración de Datos del Censo para TEST ---")
# 1.1 Cargar y concatenar CSVs del censo de NL y TAM
try:
    censo_nl_df = pd.read_csv(ARCHIVO_CENSO_NL, dtype={COL_CENSO_SCITEL_ENTIDAD:str, COL_CENSO_SCITEL_MUNICIPIO:str, COL_CENSO_SCITEL_LOCALIDAD:str, COL_CENSO_SCITEL_AGEB:str})
    censo_tam_df = pd.read_csv(ARCHIVO_CENSO_TAM, dtype={COL_CENSO_SCITEL_ENTIDAD:str, COL_CENSO_SCITEL_MUNICIPIO:str, COL_CENSO_SCITEL_LOCALIDAD:str, COL_CENSO_SCITEL_AGEB:str})
    censo_ageb_df_completo = pd.concat([censo_nl_df, censo_tam_df], ignore_index=True)
    print(f"Datos censales de NL y TAM cargados y concatenados: {len(censo_ageb_df_completo)} registros AGEB.")
except Exception as e:
    print(f"Error al cargar/concatenar datos del censo: {e}")
    raise

In [ ]:
# 1.2 Construir CVEGEO en datos del censo
try:
    censo_ageb_df_completo[COL_CENSO_SCITEL_ENTIDAD] = censo_ageb_df_completo[COL_CENSO_SCITEL_ENTIDAD].str.zfill(2)
    censo_ageb_df_completo[COL_CENSO_SCITEL_MUNICIPIO] = censo_ageb_df_completo[COL_CENSO_SCITEL_MUNICIPIO].str.zfill(3)
    censo_ageb_df_completo[COL_CENSO_SCITEL_LOCALIDAD] = censo_ageb_df_completo[COL_CENSO_SCITEL_LOCALIDAD].str.zfill(4)
    censo_ageb_df_completo[COL_CENSO_SCITEL_AGEB] = censo_ageb_df_completo[COL_CENSO_SCITEL_AGEB].astype(str).str.upper().str.strip() # .str.pad(width=4, side='left', fillchar='0')

    censo_ageb_df_completo[COL_CENSO_CVEGEO_CONSTRUIDA] = (
        censo_ageb_df_completo[COL_CENSO_SCITEL_ENTIDAD] +
        censo_ageb_df_completo[COL_CENSO_SCITEL_MUNICIPIO] +
        censo_ageb_df_completo[COL_CENSO_SCITEL_LOCALIDAD] +
        censo_ageb_df_completo[COL_CENSO_SCITEL_AGEB]
    )
    print(f"Columna '{COL_CENSO_CVEGEO_CONSTRUIDA}' creada en datos del censo.")
    censo_para_merge_df = censo_ageb_df_completo.drop_duplicates(subset=[COL_CENSO_CVEGEO_CONSTRUIDA])
except Exception as e:
    print(f"Error construyendo CVEGEO en datos del censo: {e}")
    raise

In [ ]:
# 1.3 Cargar shapefile nacional de AGEBs
try:
    ageb_gdf_nacional_shp = gpd.read_file(SHAPEFILE_AGEB_NACIONAL)
    if ageb_gdf_nacional_shp.crs is None: ageb_gdf_nacional_shp.set_crs(CRS_SHAPEFILE_AGEB, inplace=True)
    elif ageb_gdf_nacional_shp.crs.to_string().upper() != CRS_SHAPEFILE_AGEB.upper():
        ageb_gdf_nacional_shp = ageb_gdf_nacional_shp.to_crs(CRS_SHAPEFILE_AGEB)
    print(f"Shapefile nacional de AGEBs cargado y con CRS {ageb_gdf_nacional_shp.crs}.")
    ageb_gdf_nacional_shp[COL_SHP_CVEGEO] = ageb_gdf_nacional_shp[COL_SHP_CVEGEO].astype(str).str.strip()
except Exception as e:
    print(f"Error cargando shapefile nacional de AGEBs: {e}")
    raise

In [ ]:
# 1.4 Unir censo a shapefile de AGEBs
try:
    ageb_con_censo_gdf_test = ageb_gdf_nacional_shp.merge(
        censo_para_merge_df, # Ya tiene solo las columnas necesarias y CVEGEO construido
        left_on=COL_SHP_CVEGEO,
        right_on=COL_CENSO_CVEGEO_CONSTRUIDA,
        how='left'
    )
    print(f"Censo unido a AGEBs nacionales. Shape resultante: {ageb_con_censo_gdf_test.shape}")
except Exception as e:
    print(f"Error uniendo censo a shapefile de AGEBs: {e}")
    raise

In [ ]:
# 1.5 Unión espacial de tiendas TEST con AGEBs+Censo
try:
    # Seleccionar columnas relevantes del censo para añadir a las tiendas
    cols_censo_en_ageb = [col for col in censo_para_merge_df.columns if col != COL_CENSO_CVEGEO_CONSTRUIDA]
    cols_para_sjoin_desde_ageb = [COL_SHP_CVEGEO] + cols_censo_en_ageb + ['geometry']
    cols_para_sjoin_desde_ageb_existentes = [c for c in cols_para_sjoin_desde_ageb if c in ageb_con_censo_gdf_test.columns]
    cols_para_sjoin_desde_ageb_existentes = list(dict.fromkeys(cols_para_sjoin_desde_ageb_existentes))
    
    ageb_para_sjoin_gdf_test = ageb_con_censo_gdf_test[cols_para_sjoin_desde_ageb_existentes]

    tiendas_test_gdf = gpd.sjoin(
        tiendas_test_gdf,
        ageb_para_sjoin_gdf_test,
        how='left',
        predicate='within' # o predicate='within'
    )
    if 'index_right' in tiendas_test_gdf.columns:
        tiendas_test_gdf = tiendas_test_gdf.drop(columns=['index_right'])
    print(f"Tiendas TEST unidas espacialmente con AGEBs+Censo. Shape: {tiendas_test_gdf.shape}")
    # Renombrar la columna CVEGEO del shapefile a 'CVEGEO' si se llamó diferente tras el sjoin
    if COL_SHP_CVEGEO + "_left" in tiendas_test_gdf.columns and COL_SHP_CVEGEO not in tiendas_test_gdf.columns :
        tiendas_test_gdf.rename(columns={COL_SHP_CVEGEO + "_left": "CVEGEO"}, inplace=True)
    elif COL_SHP_CVEGEO + "_right" in tiendas_test_gdf.columns and COL_SHP_CVEGEO not in tiendas_test_gdf.columns:
         tiendas_test_gdf.rename(columns={COL_SHP_CVEGEO + "_right": "CVEGEO"}, inplace=True)
    elif COL_SHP_CVEGEO not in tiendas_test_gdf.columns and COL_CENSO_CVEGEO_CONSTRUIDA in tiendas_test_gdf.columns: # Si tomó la del censo
        tiendas_test_gdf.rename(columns={COL_CENSO_CVEGEO_CONSTRUIDA: "CVEGEO"}, inplace=True)
        
except Exception as e:
    print(f"Error en unión espacial de tiendas TEST con AGEBs+Censo: {e}")
    raise


In [ ]:
# === PASO 2: Integración de Datos del DENUE (similar a 05_...) ===
print("\n--- Iniciando Integración de Datos del DENUE para TEST ---")
# 2.1 Cargar diccionario SCIAN
try:
    with open(JSON_CATEGORIAS_SCIAN, 'r', encoding='utf-8') as f:
        categorias_pdi_scian_dict = json.load(f)
    print("Diccionario de categorías SCIAN cargado.")
except Exception as e:
    print(f"Error cargando JSON de categorías SCIAN: {e}")
    raise

In [ ]:
# 2.2 Cargar y preparar DENUE GDF
denue_dfs_list_test = []
for archivo_csv in ARCHIVOS_DENUE_CSV_LISTA:
    try:
        df_temp = pd.read_csv(archivo_csv, low_memory=False, encoding='latin1')
        denue_dfs_list_test.append(df_temp)
    except Exception as e: print(f"Error cargando DENUE CSV '{archivo_csv}': {e}")
if not denue_dfs_list_test: raise FileNotFoundError("No se cargaron archivos DENUE para TEST.")
denue_df_completo_test = pd.concat(denue_dfs_list_test, ignore_index=True)

denue_df_completo_test[COL_DENUE_LAT] = pd.to_numeric(denue_df_completo_test[COL_DENUE_LAT], errors='coerce')
denue_df_completo_test[COL_DENUE_LON] = pd.to_numeric(denue_df_completo_test[COL_DENUE_LON], errors='coerce')
denue_df_completo_test.dropna(subset=[COL_DENUE_LAT, COL_DENUE_LON], inplace=True)
denue_df_completo_test[COL_DENUE_SCIAN_CODE] = denue_df_completo_test[COL_DENUE_SCIAN_CODE].astype(str).str.strip()

denue_gdf_test = gpd.GeoDataFrame(
    denue_df_completo_test,
    geometry=gpd.points_from_xy(denue_df_completo_test[COL_DENUE_LON], denue_df_completo_test[COL_DENUE_LAT]),
    crs=CRS_GEOGRAFICO_WGS84
)
if denue_gdf_test.crs.to_string().upper() != CRS_PROYECTADO_OBJETIVO.upper():
    denue_gdf_test = denue_gdf_test.to_crs(CRS_PROYECTADO_OBJETIVO)
print(f"DENUE para TEST preparado y reproyectado. Shape: {denue_gdf_test.shape}, CRS: {denue_gdf_test.crs}")

In [ ]:
# 2.3 Filtrar DENUE y calcular características
gdfs_pdi_denue_filtrados_test = {}
for categoria, scian_codes_list in categorias_pdi_scian_dict.items():
    if not scian_codes_list: 
        gdfs_pdi_denue_filtrados_test[categoria] = gpd.GeoDataFrame(columns=denue_gdf_test.columns, geometry=[], crs=denue_gdf_test.crs)
        continue
    scian_codes_str = [str(code).strip() for code in scian_codes_list]
    gdfs_pdi_denue_filtrados_test[categoria] = denue_gdf_test[denue_gdf_test[COL_DENUE_SCIAN_CODE].isin(scian_codes_str)]

In [ ]:
# 2.3.A Contar PDIs DENUE en Radios
for radio_m in RADIOS_M:
    tiendas_buffer_gdf_test = tiendas_test_gdf.copy()
    tiendas_buffer_gdf_test['geometry_buffer'] = tiendas_test_gdf.geometry.buffer(radio_m)
    tiendas_buffer_gdf_test = tiendas_buffer_gdf_test.set_geometry('geometry_buffer')
    for categoria, pdi_gdf_cat in tqdm(gdfs_pdi_denue_filtrados_test.items(), desc=f"DENUE Conteo Test (R{radio_m}m)"):
        nombre_col = f'denue_conteo_{categoria}_{radio_m}m'
        if not pdi_gdf_cat.empty:
            try: joined = gpd.sjoin(tiendas_buffer_gdf_test[['TIENDA_ID', 'geometry_buffer']], pdi_gdf_cat[['geometry']], how='left', predicate='intersects')
            except TypeError: joined = gpd.sjoin(tiendas_buffer_gdf_test[['TIENDA_ID', 'geometry_buffer']], pdi_gdf_cat[['geometry']], how='left', op='intersects')
            conteo = joined.dropna(subset=['index_right']).groupby('TIENDA_ID').size()
            tiendas_test_gdf[nombre_col] = tiendas_test_gdf['TIENDA_ID'].map(conteo).fillna(0).astype(int)
        else: tiendas_test_gdf[nombre_col] = 0

In [ ]:
# 2.3.B Distancia al PDI DENUE más Cercano
for categoria, pdi_gdf_cat in tqdm(gdfs_pdi_denue_filtrados_test.items(), desc="DENUE Dist Test"):
    nombre_col = f'denue_dist_{categoria}_cercano_m'
    if not pdi_gdf_cat.empty and not pdi_gdf_cat.geometry.is_empty.all():
        try:
            sindex_pdi_cat = pdi_gdf_cat.sindex
            distancias = []
            for _, tienda_row in tiendas_test_gdf.iterrows():
                tienda_geom = tienda_row.geometry
                pos_indices = list(sindex_pdi_cat.intersection(tienda_geom.buffer(max(RADIOS_M) * 5).bounds)) # Buffer amplio
                if not pos_indices: distancias.append(np.nan); continue
                candidatos = pdi_gdf_cat.iloc[pos_indices]
                if candidatos.empty: distancias.append(np.nan); continue
                distancias.append(candidatos.geometry.distance(tienda_geom).min())
            tiendas_test_gdf[nombre_col] = distancias
            tiendas_test_gdf[nombre_col] = tiendas_test_gdf[nombre_col].fillna(99999).astype(float)
        except Exception: # Fallback
            tiendas_test_gdf[nombre_col] = tiendas_test_gdf.geometry.progress_apply(lambda g: pdi_gdf_cat.geometry.distance(g).min() if not pdi_gdf_cat.empty else np.nan).fillna(99999).astype(float)
    else: tiendas_test_gdf[nombre_col] = 99999.0
print("Características DENUE añadidas a tiendas_test_gdf.")

In [ ]:
# === PASO 3: Integración de Datos de OSM (Completo) ===
print("\n--- Iniciando Integración de Datos de OSM para TEST ---")
# 3.1 Cargar y preparar OSM POIs y Roads (similar al notebook 06)
try:
    osm_pois_gdf_raw_test = gpd.read_file(SHP_OSM_POIS_FILE)
    osm_pois_gdf_test = osm_pois_gdf_raw_test.to_crs(CRS_PROYECTADO_OBJETIVO)
    osm_roads_gdf_raw_test = gpd.read_file(SHP_OSM_ROADS_FILE)
    osm_roads_gdf_test = osm_roads_gdf_raw_test.to_crs(CRS_PROYECTADO_OBJETIVO)
    print(f"OSM POIs y Roads para TEST preparados y reproyectados. Shapes: POIs {osm_pois_gdf_test.shape}, Roads {osm_roads_gdf_test.shape}")
except Exception as e: print(f"Error cargando datos OSM para TEST: {e}"); raise

In [ ]:
# 3.2 Filtrar POIs OSM y calcular características (conteo)
# Usar `categorias_pois_osm` del notebook 06 (debes definirla aquí o importarla)
categorias_pois_osm_dict = {
    'paradas_autobus': ['bus_stop', 'bus_station'],
    'estaciones_tren_metro': ['railway_station', 'subway_entrance', 'tram_stop', 'halt'],
    'atm_cajeros': ['atm'],
    'bancos_osm': ['bank'], # Puede solaparse con DENUE, pero OSM puede tener más ATMs
    'parques_recreacion': ['park', 'playground', 'sports_centre', 'pitch', 'leisure_centre'],
    'escuelas_osm': ['school', 'kindergarten'], # OSM puede tener guarderías ('kindergarten')
    'universidades_colegios_osm': ['university', 'college']
    # Añade más si son relevantes y OSM los mapea bien (ej. 'fuel' para gasolineras si DENUE no fue suficiente)
}
gdfs_pois_osm_filtrados_test = {}
for categoria, fclass_values in categorias_pois_osm_dict.items():
    gdfs_pois_osm_filtrados_test[categoria] = osm_pois_gdf_test[osm_pois_gdf_test[COL_OSM_POI_FCLASS].isin(fclass_values)]

for radio_m in RADIOS_M: # Usar los mismos radios que para DENUE, o ajustar
    tiendas_buffer_gdf_test = tiendas_test_gdf.copy()
    tiendas_buffer_gdf_test['geometry_buffer'] = tiendas_test_gdf.geometry.buffer(radio_m)
    tiendas_buffer_gdf_test = tiendas_buffer_gdf_test.set_geometry('geometry_buffer')
    for categoria, pdi_gdf_cat in tqdm(gdfs_pois_osm_filtrados_test.items(), desc=f"OSM POIs Conteo Test (R{radio_m}m)"):
        nombre_col = f'osm_conteo_{categoria}_{radio_m}m'
        if not pdi_gdf_cat.empty:
            try: joined = gpd.sjoin(tiendas_buffer_gdf_test[['TIENDA_ID', 'geometry_buffer']], pdi_gdf_cat[['geometry']], how='left', predicate='intersects')
            except TypeError: joined = gpd.sjoin(tiendas_buffer_gdf_test[['TIENDA_ID', 'geometry_buffer']], pdi_gdf_cat[['geometry']], how='left', op='intersects')
            conteo = joined.dropna(subset=['index_right']).groupby('TIENDA_ID').size()
            tiendas_test_gdf[nombre_col] = tiendas_test_gdf['TIENDA_ID'].map(conteo).fillna(0).astype(int)
        else: tiendas_test_gdf[nombre_col] = 0
print("Características de conteo de POIs OSM añadidas a tiendas_test_gdf.")

In [ ]:
# 3.3 Calcular características de Red Vial OSM (distancia a vía principal)
if osm_roads_gdf_test is not None and not osm_roads_gdf_test.empty:
    tipos_via_principal_osm = ['motorway', 'trunk', 'primary', 'secondary'] # Ejemplo
    vias_principales_osm_gdf_test = osm_roads_gdf_test[osm_roads_gdf_test[COL_OSM_ROAD_FCLASS].isin(tipos_via_principal_osm)]
    if not vias_principales_osm_gdf_test.empty:
        try:
            sindex_vias_osm = vias_principales_osm_gdf_test.sindex
            dist_via_osm = []
            for _, tienda_row in tqdm(tiendas_test_gdf.iterrows(), total=len(tiendas_test_gdf), desc="OSM Dist Vía Test"):
                tienda_geom = tienda_row.geometry
                pos_indices = list(sindex_vias_osm.intersection(tienda_geom.buffer(max(RADIOS_M) * 10).bounds)) # Buffer de búsqueda más grande
                if not pos_indices: dist_via_osm.append(np.nan); continue
                candidatas = vias_principales_osm_gdf_test.iloc[pos_indices]
                if candidatas.empty: dist_via_osm.append(np.nan); continue
                dist_via_osm.append(candidatas.geometry.distance(tienda_geom).min())
            tiendas_test_gdf['osm_dist_via_principal_m'] = dist_via_osm
            tiendas_test_gdf['osm_dist_via_principal_m'] = tiendas_test_gdf['osm_dist_via_principal_m'].fillna(99999).astype(float)
        except Exception: # Fallback
            tiendas_test_gdf['osm_dist_via_principal_m'] = tiendas_test_gdf.geometry.progress_apply(lambda g: vias_principales_osm_gdf_test.geometry.distance(g).min() if not vias_principales_osm_gdf_test.empty else np.nan).fillna(99999).astype(float)
    else: tiendas_test_gdf['osm_dist_via_principal_m'] = 99999.0
else: tiendas_test_gdf['osm_dist_via_principal_m'] = 99999.0
print("Característica de distancia a vía principal OSM añadida a tiendas_test_gdf.")


In [ ]:
# === PASO 4: Integración de Datos del IMU (Completo) ===
print("\n--- Iniciando Integración de Datos del IMU para TEST ---")
try:
    # Cargar IMU CSV (esto ya lo tienes bien con utf-8-sig)
    imu_df_raw_test = pd.read_csv(ARCHIVO_IMU_CSV, encoding='utf-8-sig') 
    print(f"IMU para TEST cargado. Shape: {imu_df_raw_test.shape}")
    
    # Preparar la clave CVEGEO del IMU
    imu_df_raw_test[COL_IMU_CVEGEO_KEY] = imu_df_raw_test[COL_IMU_CVEGEO_KEY].astype(str).str.strip()
    
    # Seleccionar solo las columnas que queremos del IMU para añadir
    cols_imu_a_seleccionar = [COL_IMU_CVEGEO_KEY, COL_IMU_INDEX_VALUE, COL_IMU_GRADE_VALUE, COL_IMU_NAT_INDEX_VALUE]
    # Verificar que todas estas columnas existan en imu_df_raw_test
    missing_imu_cols = [col for col in cols_imu_a_seleccionar if col not in imu_df_raw_test.columns]
    if missing_imu_cols:
        raise ValueError(f"Columnas IMU para seleccionar faltantes: {missing_imu_cols}. Disponibles: {imu_df_raw_test.columns.tolist()}")
    
    imu_para_merge_df_test = imu_df_raw_test[cols_imu_a_seleccionar].drop_duplicates(subset=[COL_IMU_CVEGEO_KEY])
    print(f"IMU para merge preparado. Shape: {imu_para_merge_df_test.shape}")

    # ANTES del merge, verificamos y eliminamos columnas conflictivas en tiendas_test_gdf
    print("\nColumnas en tiendas_test_gdf ANTES del merge con IMU:", tiendas_test_gdf.columns.tolist())
    columnas_base_imu = ['IM_2020', 'GM_2020', 'IMN_2020'] # Nombres base sin sufijos
    columnas_a_eliminar_de_tiendas = []

    for col_base in columnas_base_imu:
        if col_base in tiendas_test_gdf.columns:
            print(f"  Eliminando columna preexistente '{col_base}' de tiendas_test_gdf.")
            columnas_a_eliminar_de_tiendas.append(col_base)
        if f"{col_base}_x" in tiendas_test_gdf.columns:
            print(f"  Eliminando columna preexistente '{col_base}_x' de tiendas_test_gdf.")
            columnas_a_eliminar_de_tiendas.append(f"{col_base}_x")
        if f"{col_base}_y" in tiendas_test_gdf.columns:
            print(f"  Eliminando columna preexistente '{col_base}_y' de tiendas_test_gdf.")
            columnas_a_eliminar_de_tiendas.append(f"{col_base}_y")
            
    if columnas_a_eliminar_de_tiendas:
        tiendas_test_gdf = tiendas_test_gdf.drop(columns=columnas_a_eliminar_de_tiendas)
        print("  Columnas conflictivas eliminadas de tiendas_test_gdf.")

    # Ahora realizamos el merge. Las columnas del IMU deberían entrar con sus nombres originales.
    tiendas_test_gdf = tiendas_test_gdf.merge(
        imu_para_merge_df_test,
        left_on='CVEGEO', 
        right_on=COL_IMU_CVEGEO_KEY, 
        how='left'
    )
    print(f"Merge con IMU completado. Shape de tiendas_test_gdf: {tiendas_test_gdf.shape}")

    tiendas_sin_imu_match = tiendas_test_gdf[COL_IMU_INDEX_VALUE].isnull().sum() 
    print(f"Número de tiendas sin datos del IMU correspondientes: {tiendas_sin_imu_match} de {len(tiendas_test_gdf)}")

    # Si la columna clave del IMU (COL_IMU_CVEGEO_KEY) se añadió y es diferente de 'CVEGEO', y es redundante, eliminarla.
    if COL_IMU_CVEGEO_KEY != 'CVEGEO' and COL_IMU_CVEGEO_KEY in tiendas_test_gdf.columns:
        tiendas_test_gdf = tiendas_test_gdf.drop(columns=[COL_IMU_CVEGEO_KEY])
        print(f"Columna clave redundante del IMU '{COL_IMU_CVEGEO_KEY}' eliminada.")
    
    print("\nColumnas en tiendas_test_gdf DESPUÉS del merge con IMU:")
    print(tiendas_test_gdf.columns.tolist()) # Verificar que IM_2020, GM_2020, IMN_2020 estén sin sufijos

except Exception as e: 
    print(f"Error en la sección de integración del IMU: {e}")
    raise

In [ ]:
# --- PASO FINAL: Guardar el GeoDataFrame de TEST Enriquecido ---
print("\n--- Guardando GeoDataFrame de TEST Enriquecido ---")
print("Primeras filas del dataset de TEST final enriquecido:")
pd.set_option('display.max_columns', None)
print(tiendas_test_gdf.head())
print("\nColumnas finales del dataset de TEST:")
print(tiendas_test_gdf.columns.tolist())

try:
    tiendas_test_gdf.to_file(ARCHIVO_SALIDA_TEST_ENRIQUECIDO_GPKG, driver='GPKG', layer=LAYER_SALIDA_TEST_ENRIQUECIDO)
    print(f"\nDataset de TEST enriquecido guardado en '{ARCHIVO_SALIDA_TEST_ENRIQUECIDO_GPKG}'")
except Exception as e: print(f"Error al guardar archivo de salida TEST: {e}"); raise

print("\n--- Enriquecimiento de DIM_TIENDA_TEST.csv Completado ---")